In [1]:
!pip install fnyzer

Defaulting to user installation because normal site-packages is not writeable

* * *

In [2]:
from fnyzer import optimize
from fnyzer import cobra2fn, optimize
from fnyzer import FNFactory

# GLPK

In [3]:
net0 = {
    'name': 'net0',
    'solver': 'glpk',  # Solver to be used
    'places': {        # Places and initial marking
        'p1': 5,       # Initial marking of p1
        'p2': None,    # The initial marking of p2 is not specified
        },
    'm0cons': ["1 <= m0['p2']", "m0['p2'] <= 2"], # Constraints for the
                                                  # initial marking of p2
    'vhandlers': {     # Event handlers
        'v1': [
            {'a': ('p1','v1'), 'b': ('v1','p2')}, # The same amount of tokens
             'a == b'                             # is consumed from p1 and
            ],                                    # produced in p2
        'v2': [
            {'a': ('p2','v2'), 'b': ('v2','p1')},
             'a == b'
            ],
        },
    'obj': {'f': "m['p1']", 'sense': 'max'}, # Objective function
    'options' : {
        'antype': 'un',        # Untimed analysis
        'xlsfile': 'net0.xls', # Results will be stored in this spreadsheet
        'netfile': 'net0.pkl', # The flexible net object with the resulting
                               # variables will be saved in this file
        }
    }

fnet = FNFactory(net0)
fnet.optimize()

# Gurobi

In [ ]:
# Use: fnyzer fnexamples guardnet
# Output: guardnet.xls, guardnet.pkl
guardnet = { 
    'name': 'guardnet',
    'solver': 'gurobi',
    'places': {'A': 10, 'B': 0},
    'trans': {'t1': {'l0': 0.0, 'a0': 0}, 't2': {'l0': 0, 'a0': 0}},
    'vhandlers': {
        'v1': [{'a': ('A','v1'), 'v': ('t1','v1')}, 'a == v'],
        'v2': [{'b': ('v2','B'), 'v': ('t2','v2')}, 'b == v'],
    },
    'regs': {
        'off': ["m['A'] >= 5"],
        'on':  ["m['A'] <= 5"],
    },
    'parts': {'Par': ['off', 'on']},
    'shandlers': {
        's1': [{'a': ('A','s1'), 'x': ('s1','t1'), 'y': ('s1','t2')},
                'x == a',
                {'off': ['y == 0.2'], 'on': ['y == a']}]
    },
    'actfplaces': ['A'],
    'exftrans': 'all',
    'actavplaces': ['A'],
    'exavtrans': 'all',
    'obj': {'f': "m['B']", 'sense':'min'},
    'options': {
        'antype' : 'mpc',
        'mpc': {
            'firstinlen': .1,
            'numsteps':   30,
            'maxnumins':  1,
            'flexins': False
        },
        'writevars': {'m': ['A', 'B'], 'l': 'all', 'L': 'all', 'U': 'all'},
        'plotres': True,
        'plotvars': {'evm':['A', 'B']}
    }
}


#model, fn = optimize(guardnet)
fnet = FNFactory(guardnet)
fnet.optimize()